In [1]:
# Import the QICK drivers and auxiliary libraries
from qick import *
%pylab inline

import xrfclk
from qick.ipq_pynq_utils import ipq_pynq_utils
# import spidev # pip package: "pip install spidev" or "apt install python3-spidev"
from qick.ipq_pynq_utils.ipq_pynq_utils import spidev # our reimplementation of spidev
from importlib.resources import open_text

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## readback of CLK104 state
You need a firmware that includes the GPIO interface to the CLK104's SPI mux. Most recent ZCU216 firmwares should have this.

In [360]:
soc = QickSoc("/data/fw/2024-12-05_216_tprocv2r21_demo/qick_216.bit", 
#              force_init_clks=True,
#              external_clk=True,
             )
soccfg = soc
print(soccfg)
if soccfg['board']=='ZCU216' and not hasattr(soc, 'clk104_gpio'):
    raise RuntimeError("this firmware does not have the CLK104 GPIO block needed for SPI readback on the ZCu216")

resetting clocks: 245.76 491.52
QICK running on ZCU216, software version 0.2.347

Firmware configuration (built Thu Dec  5 14:57:40 2024):

	Global clocks (MHz): tProc dispatcher timing 614.400, RF reference 245.760
	Groups of related clocks: [tProc timing clock, DAC tile 2], [DAC tile 3], [ADC tile 2]

	6 signal generator channels:
	0:	axis_signal_gen_v6 - fs=9830.400 Msps, fabric=614.400 MHz
		envelope memory: 16384 complex samples (1.667 us)
		32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_sg_mux8_v1 - fs=9830.400 Msps, fabric=614.400 MHz
		32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_sg_int4_v2 - fs=6881.280 Msps, fabric=430.080 MHz
		envelope memory: 16384 complex samples (38.095 us)
		32-bit DDS, range=1720.320 MHz
		DAC tile 3, blk 0 is 0_231, on JHC3
	3:	axis_sg_int4_v2 - fs=6881.280 Msps, fabric=430.080 MHz
		envelope memory: 16384 complex samples (38.095 us)
		32-bit DDS, range=1720.320 MHz
		DAC tile 3, blk 1 is

In [300]:
# some generic register read/write functions
def write_reg(spi, regdev, regvals):
    for k,v in regvals.items():
        getattr(regdev, k).value = v
        
    for addr in regdev.find_addrs(regvals.keys()):
        msg = regdev.registers_by_addr[addr].get_raw().to_bytes(length=3, byteorder='big')
        spi.writebytes(msg)

def read_reg(spi, dummydev, regnames):
    for addr in dummydev.find_addrs(regnames):
        msg = (0x800000 + (addr<<dummydev.dw)).to_bytes(length=3, byteorder='big')
#         print(msg)
        a = spi.xfer(msg)
#         print(a)
        res = int.from_bytes(a, byteorder='big')

        dummydev.registers_by_addr[addr].parse(res)

    readbacks = {}
    for regname in regnames:
        readbacks[regname] = getattr(dummydev, regname).value
    return readbacks

In [357]:
# read the LMK04828's readback registers, clear the lock-lost flags, and print a description of the status
def check_CLK104(soc):
    dummy_lmk = ipq_pynq_utils.clock_models.LMK04828B(10, 10, 10, 160)
    regs_lmk = ipq_pynq_utils.clock_models.LMK04828B(10, 10, 10, 160)
    with open_text("xrfclk", "LMK04828_245.76.txt") as f:
        regs_lmk.init_from_file(f)

    soc.clk104_gpio.channel1.setdirection('out')
    soc.clk104_gpio.channel1.write(0x2, 0xff)
    with spidev.SpiDev(1, 1) as spi:

        spi.bits_per_word = 8
        spi.max_speed_hz = 100000

        # set SPI output type to push-pull; default is 6 (open-drain) which is incorrect
        write_reg(spi, regs_lmk, {"PLL1_LD_TYPE": 3})

        readbacks = read_reg(spi, dummy_lmk, ['RB_DAC_VALUE', 
                                        'RB_CLKin0_LOS', 
                                        'RB_CLKin1_LOS', 
                                        'RB_PLL1_LD', 
                                        'RB_PLL2_LD', 
                                        'RB_PLL1_LD_LOST', 
                                        'RB_PLL2_LD_LOST'])

    #     for k,v in readbacks.items():
    #         print(k,v)

        # clear the lock-lost flags
        write_reg(spi, regs_lmk, {"CLR_PLL1_LD_LOST": 1,
                                 "CLR_PLL2_LD_LOST": 1})
        write_reg(spi, regs_lmk, {"CLR_PLL1_LD_LOST": 0,
                                 "CLR_PLL2_LD_LOST": 0})


        for ch, name in [(1, 'PLL1 (input PLL)'), (2, 'PLL2 (loop PLL)')]:
            if readbacks['RB_PLL%d_LD'%(ch)] == 1:
                if readbacks['RB_PLL%d_LD_LOST'%(ch)] == 0:
                    print("%s lock is locked, and has held lock since the last time you checked"%(name))
                else:
                    print("%s is locked, but has lost lock since the last time you checked"%(name))
            else:
                print("%s not locked"%(name))


Run this repeatedly to monitor the status of the two PLL stages inside the LMK04828 chip.

The first time you run this after initializing clocks, it will tell you that both PLLs are locked, but have lost lock. Further checks will report steady lock.

In [364]:
check_CLK104(soc)

PLL1 (input PLL) lock is locked, and has held lock since the last time you checked
PLL2 (loop PLL) lock is locked, and has held lock since the last time you checked


## scratchwork

In [2]:
# # how to use xrfclk to set clocks

# xrfclk.set_ref_clks(lmk_freq=245.76, lmx_freq=245.76*2)

In [5]:
# how to use IPQ utils to set clocks

board = ipq_pynq_utils.ZCU208Board()

regs_lmk = ipq_pynq_utils.clock_models.LMK04828B(10, 10, 10, 160)
with open_text("xrfclk", "LMK04828_245.76.txt") as f:
    regs_lmk.init_from_file(f)
regs_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
with open_text("xrfclk", "LMX2594_491.52.txt") as f:
    regs_lmx.init_from_file(f)
    
ipq_pynq_utils.ZCU208Board._write_registers(board.spi_lmk, [0x90] + regs_lmk.get_register_dump())
ipq_pynq_utils.ZCU208Board._write_registers(board.spi_adc, 
                                            [0x700000, 0x6f0000, 0x6e0000] + regs_lmx.get_register_dump())
ipq_pynq_utils.ZCU208Board._write_registers(board.spi_dac, 
                                            [0x700000, 0x6f0000, 0x6e0000] + regs_lmx.get_register_dump())

In [291]:
def make_lmk_regs():
    """This reproduces the LMK04828_245.76.txt register file from xrfclk.
    """
    regs = ipq_pynq_utils.clock_models.LMK04828B(10, 10, 10, 160)

    regs.CLKin0_R.value = 125
    regs.CLKin1_R.value = 125
    regs.CLKin2_R.value = 960
    regs.CLKin_SEL0_TYPE.value = 6
    regs.CLKin_SEL1_TYPE.value = 6
    regs.CLKin_SEL_MODE.value = 1

    regs.DAC_CLK_MULT.value = 3
    regs.FB_MUX.value = 1
    regs.HOLDOVER_PLL1_DET.value = 1
    regs.MAN_DAC_EN.value = 1
    regs.OPT_REG_1.value = 21
    regs.OPT_REG_2.value = 51
    regs.OSCin_FREQ.value = 1
    regs.OSCout_FMT.value = 0

    regs.PLL1_CP_GAIN.value = 10
    regs.PLL1_LD_MUX.value = 7
    regs.PLL1_N.value = 2000
    regs.PLL2_LD_MUX.value = 3
    regs.PLL2_LD_TYPE.value = 3
    regs.PLL2_N_CAL.value = 160

    regs.RESET_TYPE.value = 6
    regs.SPI_3WIRE_DIS.value = 1

    for i in range(0,14,2):
        getattr(regs, 'SYNC_DIS%d'%(i)).value = 1
    regs.SYNC_DISSYSREF.value = 1
    regs.SYNC_MODE.value = 1
    regs.SYNC_POL.value = 1

    regs.SYSREF_DDLY.value = 1
    regs.SYSREF_DDLY_PD.value = 0
    regs.SYSREF_GBL_PD.value = 1
    regs.SYSREF_MUX.value = 3
    regs.SYSREF_PD.value = 0

    regs.TRACK_EN.value = 0

    regs.set_refclk(2457.6)
    regs.set_sysref(7.68)

    # DOut0: RF_PLL_ADC_REF 
    # SDOut1: RF_PLL_ADC SYNC
    # SDOut3: AMS_SYSREF
    # DOut4: RF_PLL_DAC_REF 
    # SDOut5: RF_PLL_DAC SYNC
    # DOut6: DAC_REFCLK
    # SDOut7: DDR_PLY_CAP
    # DOut8: PL_CLK
    # SDOut9: PL_SYSREF
    # SDOut11: EXT_REF_OUT
    # DOut12: ADC_REFCLK

    for i in range(0,14,2):
        getattr(regs, 'CLKout%d_%d_IDL'%(i, i+1)).value = 1
        getattr(regs, 'CLKout%d_%d_ODL'%(i, i+1)).value = 1
        getattr(regs, 'SDCLKout%d_DDLY'%(i+1)).value = 1
    for i in [2, 10]:
        getattr(regs, 'DCLKout%d_DDLY_PD'%(i)).value = 1
    for i in [0, 4, 6, 12]:
        getattr(regs, 'SDCLKout%d_DIS_MODE'%(i+1)).value = 1
    for i, branch in enumerate(regs.clock_branches):
        branch.CLK_PD.value = 0
        if i in [0, 2, 3, 4, 6]: 
            branch.DCLK_MUX.value = 1
            branch.DCLK_FMT.value = 3
        branch.SDCLK_MUX.value = 1
        if i in [0, 2, 6]: branch.SDCLK_PD.value = 1
        if i in [1, 4, 5]: branch.SDCLK_FMT.value = 3
        if i == 4:
            branch.request_freq(122.88)
        else:
            branch.request_freq(245.76)
    return regs

regs_lmk = make_lmk_regs()
# regs_lmk.update(printDebug=True)

In [294]:
from qick.ipq_pynq_utils.ipq_pynq_utils.clock_models import *

def compare_regs(regs_theirs, regs_ours):
    dump_theirs = regs_theirs.get_register_dump()
    dump_ours = regs_ours.get_register_dump()

    n_diffs = 0
    for i in range(len(dump_theirs)):
        if dump_theirs[i] != dump_ours[i]:
            n_diffs += 1
            print(hex(dump_theirs[i]), hex(dump_ours[i]))
    print(n_diffs)

    regnames = []
    for k,v in regs_theirs.__dict__.items():
        if isinstance(v, (MultiRegister, Field)):
            regnames.append(k)

    changes = []
    for k in regnames:
        val_theirs = getattr(regs_theirs, k).value
        if getattr(regs_ours, k).value != val_theirs:
            changes.append((k, val_theirs))
    changes.sort()

    for k,v in changes:
        print(k, v)
    print(len(changes))

regs_lmk_xrfclk = ipq_pynq_utils.clock_models.LMK04828B(10, 10, 10, 160)
with open_text("xrfclk", "LMK04828_245.76.txt") as f:
    regs_lmk_xrfclk.init_from_file(f)

compare_regs(regs_lmk_xrfclk, regs_lmk)

0x4d0 0x404
0x55b 0x505
0x600 0x620
3
ID_MASKREV 0
1


In [118]:
# # to see what fields are in a given register:
# regs_lmk.registers_by_addr[0x10F]

{'addr': 271, 'fields': [{'end': 7, 'start': 7, 'width': 1, 'mask': 128, 'name': 'SDCLKout3_POL', 'description': 'Sets the polarity of clock on SDCLKout3 when device clock output is selected with SDCLKout3_MUX.', '__doc__': 'Sets the polarity of clock on SDCLKout3 when device clock output is selected with SDCLKout3_MUX.', 'default': 0, 'value': 0, 'enum_map': {0: NORMAL, 1: INVERTED}, 'valid_type': 'enum', 'NORMAL': NORMAL, 'INVERTED': INVERTED}, {'end': 6, 'start': 4, 'width': 3, 'mask': 112, 'name': 'SDCLKout3_FMT', 'description': 'Sets the output format of the SYSREF clocks', '__doc__': 'Sets the output format of the SYSREF clocks', 'default': 0, 'value': 3, 'enum_map': {0: POWERDOWN, 1: LVDS, 2: HSDS_6_MA, 3: HSDS_8_MA, 4: HSDS_10_MA, 5: LVPECL_1_6_V, 6: LVPECL_2_0_V, 7: LCPECL}, 'valid_type': 'enum', 'POWERDOWN': POWERDOWN, 'LVDS': LVDS, 'HSDS_6_MA': HSDS_6_MA, 'HSDS_8_MA': HSDS_8_MA, 'HSDS_10_MA': HSDS_10_MA, 'LVPECL_1_6_V': LVPECL_1_6_V, 'LVPECL_2_0_V': LVPECL_2_0_V, 'LCPECL': L

In [330]:
regs_lmk.LOS_EN.value

0

In [334]:
regs_lmk.CLKin0_TYPE.value

0

PLL1 (input PLL) not locked
PLL2 (loop PLL) lock is locked, and has held lock since the last time you checked


In [ ]:
dummy_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
regs_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
with open_text("xrfclk", "LMX2594_491.52.txt") as f:
    regs_lmx.init_from_file(f)

soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x0, 0xff)

with spidev.SpiDev(1, 3) as spi:
    spi.bits_per_word = 8
    spi.max_speed_hz = 100000

    # set output to SPI readback; default is 1 (lock detect) which drives the LED
    write_reg(spi, regs_lmx, {"MUXOUT_LD_SEL": 0})
    readbacks = read_reg(spi, dummy_lmx, ['rb_LD_VTUNE',
                                    'rb_VCO_SEL',
                                    'rb_VCO_CAPCTRL',
                                    'rb_VCO_DACISET'])
    
    for k,v in readbacks.items():
        print(k,v)
        
    # switch back to indicating lock-detect with the LED
    write_reg(spi, regs_lmx, {"MUXOUT_LD_SEL": 1})

In [ ]:
dummy_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
regs_lmx = ipq_pynq_utils.clock_models.LMX2594(245.76)
with open_text("xrfclk", "LMX2594_491.52.txt") as f:
    regs_lmx.init_from_file(f)

soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x1, 0xff)

with spidev.SpiDev(1, 2) as spi:
    spi.bits_per_word = 8
    spi.max_speed_hz = 100000

    # set output to SPI readback; default is 1 (lock detect) which drives the LED
    write_reg(spi, regs_lmx, {"MUXOUT_LD_SEL": 0})
    readbacks = read_reg(spi, dummy_lmx, ['rb_LD_VTUNE',
                                    'rb_VCO_SEL',
                                    'rb_VCO_CAPCTRL',
                                    'rb_VCO_DACISET'])
    
    for k,v in readbacks.items():
        print(k,v)
        
    # switch back to indicating lock-detect with the LED
    write_reg(spi, regs_lmx, {"MUXOUT_LD_SEL": 1})
    

In [ ]:
# print(ipq_lmk.PLL1_LD_MUX)
# print(ipq_lmk.PLL1_LD_TYPE)
# print(ipq_adc.MUXOUT_LD_SEL)

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x2, 0xff)
s = board.spi_lmk

addr = 388
msg = [((addr>>8) & 0xFF) + (1<<7), addr & 0xFF, 0]
print(msg)
a = s.xfer(msg)
print(hex(a[2]))

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x0, 0xff)

s = board.spi_adc
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer(msg)
print(hex((a[1] << 8)+a[2]))

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x1, 0xff)

s = board.spi_dac
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer(msg)
print(a)

In [ ]:
for k,v in dummy_lmk.registers_by_addr.items():
    for x in v.fields:
        if x.name != 'CONST':
            print(k, x.name)

In [ ]:
dir(dummy_lmk)

In [ ]:
# for x in dummy_reg.fields:
#     print(x.name, x.value)

In [ ]:
# dummy_lmk.registers_by_addr[0x182].fields[0].name

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x2, 0xff)
s = board.spi_lmk

for addr in [0x182, 0x183, 0x184, 0x185]:
    dummy_reg = dummy_lmk.registers_by_addr[addr]
    # print(dummy_reg)
    # for x in dummy_reg.fields:
    #     print(x.name, x.value)
    
    msg = [((addr>>8) & 0xFF) + (1<<7), addr & 0xFF, 0]
#     print(msg)
    a = s.xfer(msg)
    res = a[2]
    print("0x%x: 0x%x"%(addr,res))
    
    dummy_reg.parse(res)
    for f in dummy_reg.fields:
        if f.name != 'CONST':
            print("%s: 0x%x"%(f.name, f.value))


In [ ]:
xrfclk.lmk_devices=[]
xrfclk.lmx_devices=[]
xrfclk.xrfclk._find_devices()


In [ ]:
xrfclk.lmx_devices

In [ ]:
xrfclk.xrfclk._read_tics_output()

In [ ]:
xrf_conf = xrfclk.xrfclk._Config['lmk04828'][245.76]
ipq_conf = [0x90] + ipq_lmk.get_register_dump()

for i in range(len(ipq_conf)):
    if i<len(xrf_conf):
        if ipq_conf[i] != xrf_conf[i]:
            print("%x\t%x"%(ipq_conf[i], xrf_conf[i]))
    else:
        print("%x"%(ipq_conf[i]))            

In [ ]:
xrf_conf = xrfclk.xrfclk._Config['lmx2594'][491.52]
ipq_conf = [0x700000, 0x6f0000, 0x6e0000] + ipq_adc.get_register_dump()

for i in range(len(ipq_conf)):
    if i<len(xrf_conf):
        if ipq_conf[i] != xrf_conf[i]:
            print("%x\t%x"%(ipq_conf[i], xrf_conf[i]))
    else:
        print("%x"%(ipq_conf[i]))            

In [ ]:
xrfclk.xrfclk._Config['lmx2594']

In [ ]:
board = ipq_pynq_utils.ZCU208Board()
board.print_clock_summary()

In [ ]:
help(s)

In [ ]:
# s = board.spi_lmk
# addr = 388

# s.writebytes([((addr>>8) & 0xFF) + (1<<7), addr & 0xFF, 0])
# a = s.readbytes(3)
# print(a)

In [ ]:
for x in soc.clk104_gpio:
    print(x)

In [ ]:
dir(soc.clk104_gpio)

In [ ]:
dir(soc.clk104_gpio.channel1)

In [ ]:
hex(soc.clk104_gpio.channel1.trimask)

In [ ]:
help(soc.clk104_gpio)

In [ ]:
soc.clk104_gpio.mmio.array[:4]

In [ ]:
soc.clk104_gpio._registers

In [ ]:
soc.clk104_gpio.channel1.setdirection('out')
soc.clk104_gpio.channel1.write(0x0, 0xff)

s = board.spi_adc
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer2(msg)
print(a)

In [ ]:
while True:
    time.sleep(0.5)
    soc.clk104_gpio.channel1.setdirection('out')
    soc.clk104_gpio.channel1.write(0x1, 0xff)

    s = board.spi_dac
    addr = 110
    msg = [addr + (1<<7), 0, 0]
    print(msg)
    a = s.xfer(msg)
    print(a)

In [ ]:
s = board.spi_adc
print(s.no_cs)
print(s.cshigh)
print(s.mode)
print(s.bits_per_word)
print(s.lsbfirst)

In [ ]:
# s.cshigh = True

In [ ]:
dir(board.spi_adc)

In [ ]:
s = board.spi_lmk
addr = 388
msg = [((addr>>8) & 0xFF) + (1<<7), addr & 0xFF, 0]
print(msg)
a = s.xfer(msg)
print(a)

s = board.spi_adc
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer3(msg)
print(a)

s = board.spi_dac
addr = 110
msg = [addr + (1<<7), 0, 0]
print(msg)
a = s.xfer3(msg)
print(a)

In [ ]:
msg

In [ ]:
clk104 = ipq_pynq_utils.clock_models.CLK104()

In [ ]:
for x in clk104.lmk.get_register_dump():
    print(hex(x))

In [ ]:
for k,v in clk104.lmk.registers_by_addr.items():
    for x in v.fields:
        if x.name != 'CONST':
            print(k, x.name)

In [ ]:
for k,v in clk104.lmx_adc.registers_by_addr.items():
    for x in v.fields:
        if x.name != 'CONST':
            print(k, x.name)

In [ ]:
help(board.spi_lmk)

In [ ]:
c = ipq_pynq_utils.clock_models.LMX2594(122.88)

In [ ]:
for x in c.get_register_dump():
    print(hex(x))

In [ ]:
c.rb_LD_VTUNE

In [ ]:
for k,v in c.registers_by_addr.items():
    for x in v.fields:
        if x.name != 'CONST':
            print(k, x.name)
#     print(k, v.fields)
    